<a href="https://colab.research.google.com/github/AguaClara/Prefabricated-Pilot-Plant/blob/master/Floc_Redesign_Workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Flocculator Redesign Workbook** 

In [0]:
!pip install aguaclara
import aguaclara.core.head_loss as minorloss
import aguaclara.design.human_access as ha
import aguaclara.core.physchem as pc
import aguaclara.core.pipes as pipes
from aguaclara.core.units import unit_registry as u
import numpy as np
import pandas as pd

Current Flocculator Redesign

In [38]:
# Governing Equation: h_l = v^2 / 2g (n_c*k_c + n_b* (Apipe/Aopen*pi_vc - 1)^2)

tot_hl = 50 * u.cm # Desired headloss for flocculator system
num_elbows = 12 # Total number of elbows in piping
K_elbow = 0.3 # headloss coefficient for 3 inch pipe 90 degree bend
vel = 1 * u.m/u.s # Average velocity through flocculator
num_baf = 48 # number of baffles in system
A_pipe = np.pi * ((pipes.ID_sch40(3*u.inch)/2) ** 2) # area of 3 inch pipe
pi_vc = 0.62 #vena contracta coefficient for 90 degree bend

A_open =  A_pipe/pi_vc * 1/((np.sqrt((2*pc.gravity*tot_hl)/(vel**2) - num_elbows*K_elbow
                         / num_baf) + 1))
percen = (A_open/A_pipe) * 100
print(A_pipe)
print(A_open)
print(percen)

7.393 inch ** 2
2.894 inch ** 2
39.15 dimensionless


In [55]:
# Round Twoski

Q_in = 1 * u.L / u.s
A_pipe = np.pi * (pipes.ID_sch40(1.5*u.inch) ** 2) # area of 3 inch pipe
pipe_vel = Q_in / A_pipe # average velocity flowing through the pipe with no baffle
print(pipe_vel.to(u.m/u.s))

tot_hl = 50 * u.cm # Desired headloss for flocculator system
num_elbows = 12 # Total number of elbows in piping
K_elbow = 0.3 # headloss coefficient for 3 inch pipe 90 degree bend
rem_hl =  (tot_hl - (num_elbows*K_elbow * (pipe_vel ** 2)/(2* pc.gravity))) #remaining headloss without elbows
print(rem_hl)

num_baf = 48 # number of baffles in system
baf_hl = rem_hl / num_baf # headloss per baffle
print(baf_hl)

targ_vc = (np.sqrt(2*pc.gravity*baf_hl/(pipe_vel ** 2)) ) #solve for vc from K equation
print(targ_vc)







0.1903 meter / second
49.34 centimeter
1.028 centimeter
-1.359 dimensionless


New Flocculator Design

     |████████████████████████████████| 81kB 3.1MB/s 
     |████████████████████████████████| 163kB 8.5MB/s 
     |████████████████████████████████| 665kB 39.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/67/cb/2a/97692914470ff85150e4769f1e143ef68c722eeacd8dccade4
  Stored in directory: /root/.cache/pip/wheels/b4/56/36/7c4d6a4e3660e233b95e3b0482551915cb184abca0f93ee1dd
  Stored in directory: /root/.cache/pip/wheels/ea/bd/5d/bcad50021b4d6e49e006edc8c271d68b4493a83e5a85551e57
Successfully built aguaclara pint onshapepy


In [0]:
##Flocculatorita Code



BAFFLE_THICKNESS = 2 * u.mm
PVC_CHANNEL_THICKNESS = 1 * u.inch

class Flocculatorita:
    """Calculates physical dimensions of a flocculatorita
    ----------------------------
    - BAFFLE_K (K or K_{baffle}): float
        - The minor loss coefficient of the flocculator baffles.
    - CHANNEL_N_MIN (N_{channel}: int
        - The minimum number of flocculator channels.
    - HS_RATIO_MIN (\Pi_{HS}): float
        - The minimum ratio between expansion height and baffle spacing
    - RATIO_MAX_HS (\Pi_{HS}): float
        - The maximum ratio between expansion height and baffle spacing
    - SDR (sdr): float
        - The standard dimension ratio.
    """

    # Increased both to provide a safety margin on flocculator head loss and
    # to simultaneously scale back on the actual collision potential we are
    # trying to achieve.
    # Originally calculated to be 2.3 from the equations:


    BAFFLE_K = 2.5
    CHANNEL_N_MIN = 1
    RATIO_MAX_HS = 6.0
    SDR = 41.0
    pi_VC_180bend=.384 #source: (Floc Design PP )

    def __init__(
            self,
            Q=1 * u.L/u.s,
            temp=25 * u.degC,
            Gt=37000,
            HL = 40 * u.cm,
            downstream_H = 2 * u.m,
            ent_tank_L=0*u.m,
            max_W=42 * u.inch,
            available_tank_diameters = [12.2, 15.1, 18.2,24.1, 30.2,36.0,42.0,47.9,59.9,72.0],
            available_obstacle_diameters = [1/2,3/4,1,5/4,3/2,2,5/2,3,7/2,4,5,6,8,10,12,14,16],
            n_channels=5,
            pi_VC_180bend=pi_VC_180bend,
            tank_diameters_prices = [[12.2, 4.07], [15.1, 5.70], [18.2, 8.00],[24.1, 12.64], [30.2, 18.60],[36.0, 24.33], [42.0, 28.40],[47.9 , 39.42] , [59.9, 63.51]],
            baffle_price = (20.5/1872) * u.USD/(u.inch**2),
            PVC_channel_price = (431/4608) * u.USD/(u.inch**2),
            obstacle_diameters_prices = [[1/2, 1.95/5],[3/4, 2.35/5],[1, 3.91/5],[5/4, 4.55/5],[3/2, 4.99/5],[2, 6.90/5],[5/2, 10.26/5],[3, 13.90/5],[7/2, 39.95/5],[4, 18.90/5],[5, 57.50/5],[6, 37.94/5],[8, 55.00/5],[10, 76.81/5],[12, 109.86/5],[14, 180.75/5],[16, 228.52/5]]
            ):

        self.Q = Q
        self.temp = temp
        self.Gt = Gt
        self.HL = HL
        self.downstream_H = downstream_H
        self.ent_tank_L = ent_tank_L
        self.max_W = max_W
        self.n_channels=n_channels
        self.pi_VC_180bend=pi_VC_180bend
        self.available_tank_diameters=available_tank_diameters
        self.tank_diameters_prices = tank_diameters_prices
        self.baffle_price = baffle_price
        self.PVC_channel_price = PVC_channel_price
        self.obstacle_diameters_prices = obstacle_diameters_prices
        self.available_obstacle_diameters = available_obstacle_diameters


    @property
    def vel_grad_avg(self):
        #NOTE: this is borrowed from AguaClara source code
        """Calculate the average velocity gradient (G-bar) of water flowing
        through the flocculator.
        :returns: Average velocity gradient (G-bar)
        :rtype: float * 1 / second
        """
        return ((u.standard_gravity * self.HL) /
               (pc.viscosity_kinematic(self.temp) * self.Gt)).to(u.s ** -1)

    @property
    def retention_time(self):
        #NOTE: this is borrowed from AguaClara source code
        """Calculates the hydraulic retention time neglecting the volume created by head loss in the flocculator.
        :returns: Hydraulic retention time (:math:`\theta`)
        :rtype: float * second
        """
        return (self.Gt / self.vel_grad_avg).to(u.s)

    @property
    def vol(self):
        #NOTE: this is borrowed from AguaClara source code
        """Calculate the target volume (not counting the volume added by head loss) of the flocculator.
        :returns: Target volume
        :rtype: float * meter ** 3
        """
        return (self.Q * self.retention_time).to(u.m ** 3)

    @property
    def cross_sectional_area(self):
        """This function calculates the necessary active cross sectional area of the flocculator"""
        return (self.vol/self.downstream_H).to(u.m**2)


    @property
    def min_diameter_tank(self):
        """"this function calculates the necessary diameter to accommodate the required cross sectional area, assuming that the only "active space" in the cross section is the largest square that will fit into the larger PVC pipe"""
        return ((2*self.cross_sectional_area)**.5).to(u.inch)

    @property
    def design_diameter_tank(self):
        """This function selects the closest available PVC tank to the calculated minimum pipe diameter"""
        for i in range(len(self.available_tank_diameters)):
            if self.available_tank_diameters[i]*u.inch <= self.min_diameter_tank:
                i +=1
            else:
                return self.available_tank_diameters[i]*u.inch

    @property
    def channel_L(self):
        """This function determines channel length based the largest square that fits inside of the selected tank. So, channel length is the the length of the square"""
        return ((self.design_diameter_tank)**2/2)**.5

    @property
    def channel_W(self):
       """This function calculates the channel width by dividing the total width of the active cross sectional area by number of channels"""
       return (self.channel_L/self.n_channels).to(u.cm)
       ###NOTE: incorporate baffle width

    @property
    def expansion_max_H(self):
        #NOTE: This is borrowed from AguaClara source code
        """"Return the maximum distance between expansions for the largest
        allowable H/S ratio.
        :returns: Maximum expansion distance
        :rtype: float * meter
        Examples
        --------
        exp_dist_max(20*u.L/u.s, 40*u.cm, 37000, 25*u.degC, 2*u.m)
        0.375 meter
        """
        return (((self.BAFFLE_K / (2 * pc.viscosity_kinematic(self.temp) * (self.vel_grad_avg ** 2))) *
                (self.Q * self.RATIO_MAX_HS / self.channel_W) ** 3) ** (1/4)).to(u.m)

    @property
    def expansion_n(self):
        #NOTE: This is borrowed from AguaClara source code
        """Return the minimum number of expansions per baffle space.
        :returns: Minimum number of expansions/baffle space
        :rtype: int
        """
        return np.ceil(self.downstream_H / self.expansion_max_H)

    @property
    def expansion_H(self):
      #NOTE: This is borrowed from AguaClara source code#NOTE: This is borrowed from AguaClara source code
        """Returns the height between flow expansions.
        :returns: Height between flow expansions
        :rtype: float * centimeter
        """
        return (self.downstream_H / self.expansion_n).to(u.cm)

    @property
    def baffle_S(self):
      #NOTE: This is borrowed from AguaClara source code
        """Return the spacing between baffles.
        :returns: Spacing between baffles
        :rtype: int
        """
        return ((self.BAFFLE_K /
                ((2 * self.expansion_H * (self.vel_grad_avg ** 2) *
                 pc.viscosity_kinematic(self.temp))).to_base_units()) ** (1/3) *
               self.Q / self.channel_W).to(u.cm)

    @property
    def obstacle_n(self):
      #NOTE: This is borrowed from AguaClara source code
        """Return the number of obstacles per baffle.
        :returns: Number of obstacles per baffle
        :rtype: int
        """
        return self.expansion_n - 1

    @property
    def obstacle_size(self):
        """This function calculates the size of obstacles, so that the obstacles cause the same flow expansion as a 180 degree bend in the flocculatorita"""

        for i in range(len(self.available_obstacle_diameters)):
            if (self.baffle_S*(1-self.pi_VC_180bend)).to(u.inch) >= self.available_obstacle_diameters[i]/2*u.inch :
                i +=1
            else:
                return self.available_obstacle_diameters[i]*u.inch

    @property
    def baffle_n(self):
        """This function calclulates the number of the baffles in the flocculator."""
        return (np.ceil((self.channel_L/self.baffle_S*self.n_channels))).to(u.dimensionless)

    @property
    def design(self):
        """Returns the designed values.
        :returns: list of designed values (G, t, channel_W, obstacle_n)
        :rtype: int
        """
        floc_dict = {'n_channels': self.n_channels,
                     'channel_L': self.channel_L,
                     'channel_W': self.channel_W,
                     'baffle_S': self.baffle_S,
                     'obstacle_n': self.obstacle_n,
                     'G': self.vel_grad_avg,
                     't': self.retention_time,
                     'expansion_max_H': self.expansion_max_H,
                     'min_diameter_tank': self.min_diameter_tank,
                     'design_diameter_tank': self.design_diameter_tank}
        return floc_dict

    @property
    def cost_of_tank(self):
        """Returns the cost of the tank.
        """
        for i in range(len(self.available_tank_diameters)):
            if self.available_tank_diameters[i]*u.inch < self.design_diameter_tank:
                i +=1
            else:
                return (self.tank_diameters_prices[i][1]*(u.USD/u.ft) * (self.downstream_H + self.HL + 10*u.cm)).to(u.USD)

    @property
    def cost_of_baffles(self):
        """Returns the cost of the baffels.
        """

        return (self.channel_W * (self.downstream_H + self.HL + (10 * u.cm)) * self.baffle_n * self.n_channels * self.baffle_price).to(u.USD)

    @property
    def cost_of_PVC_channels(self):
        """Returns the cost of the PVC Channels.
        """
        return ((self.n_channels+1) * (self.downstream_H + self.HL + 10 * u.cm) * self.channel_L * self.PVC_channel_price).to(u.USD)


    @property
    def cost_of_obstacles(self):
        """Returns the cost of the obstacles.
        """
        for i in range(len(self.available_obstacle_diameters)):
          if self.available_obstacle_diameters[i]*u.inch < self.obstacle_size:
            i +=1
          else:
            return (self.obstacle_diameters_prices[i][1]*(u.USD/u.ft) * self.obstacle_n*self.baffle_n*self.n_channels*self.channel_W/2).to(u.USD)

    @property
    def total_cost(self):
        """Returns the cost total cost.
        """
        return self.cost_of_tank + self.cost_of_baffles + self.cost_of_PVC_channels + self.cost_of_obstacles

    @property
    def cost(self):
        """Returns costs of of everything
        """
        cost_dict = {'cost of tank':self.cost_of_tank,
                     'cost of baffles':self.cost_of_baffles,
                     'cost of PVC channels':self.cost_of_PVC_channels,
                     'cost of halfpipes':self.cost_of_obstacles,
                     'total cost':self.total_cost}
        return cost_dict


    @property
    def pi_sedtank_floc(self):
        """This function calculates the ratio of the sed tank diameter to the flocculator diameter if the sed tank's up flow velocity is 1 mm/s and height is 2 m"""
        up_V=1*u.mm/u.s
        depth=2*u.m
        A_sedtank=self.Q/up_V
        d_sedtank=(2*A_sedtank**.5).to(u.inch)
        return (d_sedtank/self.min_diameter_tank)


##make a data frame to show results
q  = 1 * u.L/u.s #change this flow rate in order to display design specs for the flocculatorita handling that flow rate
tester=Flocculatorita(Q=q)

data ={'':['Tank Pipe Diameter', 'Width of Channels', 'Baffle Spacing', 'Cost'],
       '1 Channel': [Flocculatorita(n_channels=1, Q=q).design_diameter_tank, Flocculatorita(n_channels=1, Q=q).channel_W, Flocculatorita(n_channels=1, Q=q).baffle_S,Flocculatorita(n_channels=1, Q=q).total_cost],
       '2 Channels': [Flocculatorita(n_channels=2, Q=q).design_diameter_tank, Flocculatorita(n_channels=2, Q=q).channel_W, Flocculatorita(n_channels=2, Q=q).baffle_S, Flocculatorita(n_channels=2, Q=q).total_cost],
       '3 Channels': [Flocculatorita(n_channels=3, Q=q).design_diameter_tank, Flocculatorita(n_channels=3, Q=q).channel_W, Flocculatorita(n_channels=3, Q=q).baffle_S,Flocculatorita(n_channels=3, Q=q).total_cost],
       '4 Channels':[Flocculatorita(n_channels=4, Q=q).design_diameter_tank, Flocculatorita(n_channels=4, Q=q).channel_W, Flocculatorita(n_channels=4, Q=q).baffle_S, Flocculatorita(n_channels=4, Q=q).total_cost],
       '5 Channels':[Flocculatorita(n_channels=5, Q=q).design_diameter_tank, Flocculatorita(n_channels=5, Q=q).channel_W, Flocculatorita(n_channels=5, Q=q).baffle_S, Flocculatorita(n_channels=5, Q=q).total_cost],
       '6 Channels':[Flocculatorita(n_channels=6, Q=q).design_diameter_tank, Flocculatorita(n_channels=6, Q=q).channel_W, Flocculatorita(n_channels=6, Q=q).baffle_S,Flocculatorita(n_channels=6, Q=q).total_cost]
       }
Flocculatorita(n_channels=2, Q=q).total_cost

df=pd.DataFrame(data)
print(df)


##Cost Comparison Code

class Floc_1_min_res(Flocculatorita):
  """This class models a flocculatorita with 1 minute residence time"""
  @property
  def retention_time(self):
    return 1*u.min
  @property
  def vol(self):
    return ((self.Q*self.retention_time).to(u.L))

"""Now, Floc_1_min_res is used to estimate the cost of the floculatorita, which would handle similar flow and have the same collision potential as the current PF300 flocculator.  """
cc=Floc_1_min_res(Q=1*u.L/u.s)
print(cc.retention_time)
print(cc.total_cost)

"""Below are the calculations for the estimate of the cost of the current PF3000 flocculator"""

three_in_pvc=(32.73*u.USD)/(10*u.ft) #this is the cost of a 10' long 3" pvc pipe from [McMaster]https://www.mcmaster.com/pvc
quantity_three_in_pvc=9*2*u.m
price_three_in_pvc=quantity_three_in_pvc*three_in_pvc ##total cost of the 3" PVC pipes that will used for the floculatorita

T_three_in_pvc=	7.66*u.USD #this is the cost from [McMaster]https://www.mcmaster.com/pvc
quantity_T_three_in_pvc=10
price_T_three_in_pvc=T_three_in_pvc*quantity_T_three_in_pvc


three_in_elbow=5.22*u.USD #this is the cost from [McMaster]https://www.mcmaster.com/pvc
quantity_three_in_elbow=12
price_three_in_elbow=three_in_elbow*quantity_three_in_elbow

pvc_sheet= 56.52*u.USD#used to make light deflectors, this is the cost from [McMaster]https://www.mcmaster.com/pvc
quantity_pvc_sheet=1
price_pvc_sheet=pvc_sheet*quantity_pvc_sheet


nuts = 9.18*u.USD #the price of one pack of 25 1/4" stainless steel nuts
quantity_nuts = 1
price_nuts = nuts * quantity_nuts

reducer_3to2 = 10.28 *u.USD #price of a 3" to 2" pvc pipe reducer from McMaster
quantity_reducer_3to2= 1
price_reducer_3to2 = reducer_3to2 * quantity_reducer_3to2

reducer_2to1 = 2.43 *u.USD #price of a 3" to 2" pvc pipe reducer from McMaster
quantity_reducer_2to1= 1
price_reducer_2to1 = reducer_2to1 * quantity_reducer_3to2

PF300cost=(price_three_in_pvc+price_T_three_in_pvc+price_three_in_elbow+price_pvc_sheet+price_nuts+price_reducer_3to2 +price_reducer_2to1).to(u.USD)



comparison=cc.total_cost/PF300cost
print('The floculatorita is', comparison, 'times the cost of the current floculatorita')


                              1 Channel  ...        5 Channels        6 Channels
0  Tank Pipe Diameter         24.1 inch  ...         24.1 inch         24.1 inch
1   Width of Channels  43.28 centimeter  ...  8.657 centimeter  7.214 centimeter
2      Baffle Spacing   2.14 centimeter  ...   7.26 centimeter  8.712 centimeter
3                Cost      987.6 dollar  ...       1932 dollar       2757 dollar

[4 rows x 7 columns]
1 minute
659.8 dollar
The floculatorita is 1.606 dimensionless times the cost of the current floculatorita
